In [2]:
import torch
from torch.nn.functional import gelu

In [14]:
X = torch.randn(4, 2, device="cpu", dtype=torch.float32)
W = torch.randn(2, 2, device="cpu", dtype=torch.float32)

In [15]:
# Column linear

W_0, W_1 = W.chunk(2, dim=1)

y_col_1 = torch.cat([gelu(X @ W_0), gelu(X @ W_1)], dim=1)

y_col_2 = gelu(torch.cat([X @ W_0, X @ W_1], dim=1))

try:
    torch.testing.assert_close(y_col_1, y_col_2, rtol=1e-5, atol=1e-5)
    col_match = True
    print("Column linear match")
except AssertionError:
    col_match = False
    print("Column linear mismatch")

y_col_1

Column linear match


tensor([[ 0.0814, -0.1499],
        [ 0.2147,  0.5063],
        [ 0.1975,  1.6052],
        [-0.1621,  1.0192]])

In [16]:
# Row linear

X_0, X_1 = X.chunk(2, dim=1)
W_0, W_1 = W.chunk(2, dim=0)

y_row_1 = gelu(X_0 @ W_0) + gelu(X_1 @ W_1)
y_row_2 = gelu(X_0 @ W_0 + X_1 @ W_1)

try:
    torch.testing.assert_close(y_row_1, y_row_2, rtol=1e-5, atol=1e-5)
    col_match = True
    print("Row linear match")
except AssertionError:
    col_match = False
    print("Row linear mismatch")

y_row_1

Row linear mismatch


tensor([[ 0.0847, -0.1884],
        [ 0.1931,  0.7320],
        [ 0.1850,  1.6781],
        [-0.1535,  0.9404]])